---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def trimstr(s,by):
    if by in s:
        return s[:s.index(by)].strip()
    return s


utowns_raw=pd.read_table('university_towns.txt',header=None)
utowns=pd.DataFrame(columns=['State','RegionName'])
    
current_state=None
current_indx=0
for indx,row in utowns_raw.iterrows():
    curstr=row[0]
    if curstr.endswith('[edit]'):
        current_state=trimstr(curstr,'[')
    else:
        current_town=trimstr(curstr,'(')
        utowns.loc[current_indx]=[current_state,current_town]
        current_indx+=1


def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''   
    
    #return pd.DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],columns=["State", "RegionName"]  )
    #return utowns[utowns['State']=='Michigan']
    return utowns

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
GDP=(pd.read_excel('gdplev.xls',skiprows=4,header=1,usecols=[4,5,6])
     .dropna()
     .rename(columns={
         'Unnamed: 4':'Quarter',
         'GDP in billions of current dollars.1':'GDP1',
         'GDP in billions of chained 2009 dollars.1':'GDP2'})
     .set_index('Quarter')
     .loc['2000q1':]
    )

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''    
    
    diffGDP1=GDP.diff(-1)
    for i in range(len(diffGDP1-2)):
        if (diffGDP1.iloc[i]['GDP1']>0) and (diffGDP1.iloc[i+1]['GDP1']>0):
            return diffGDP1.iloc[i].name
    return ''

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    diffGDP2=GDP.diff(1)
    for i in range(2,len(diffGDP2-2)):
        if (diffGDP2.iloc[i-2]['GDP1']<0) and (diffGDP2.iloc[i-1]['GDP1']<0) \
        and (diffGDP2.iloc[i]['GDP1']>0) and (diffGDP2.iloc[i+1]['GDP1']>0):
            return diffGDP2.iloc[i+1].name
    return ''

get_recession_end()
#diffGDP2.iloc[10:]

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    rstart=get_recession_start()
    rend=get_recession_end()
    #return GDP.loc[rstart:rend]
    return GDP[GDP['GDP1']==np.min(GDP.loc[rstart:rend]['GDP1'])].index[-1]

get_recession_bottom()

'2009q2'

In [7]:
house_raw=pd.read_csv('City_Zhvi_AllHomes.csv').set_index('RegionID')
house_raw['State']=house_raw['State'].map(states)
house_raw=house_raw.sort_values('State').set_index(['State','RegionName'])
#print(list(GDP.index.values))
quarterlst=list(GDP.index.values)
quarterlst.append('2016q3')
start=48
for cq in quarterlst:
    house_raw[cq]=house_raw.iloc[:,start:start+3].mean(axis=1)
    start+=3
house=house_raw.iloc[:,-67:]

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    ''' 
    return house

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                       
Alabama   Mulga                        NaN            NaN            NaN   
          Trussville         164400.000000  165900.000000  167600.000000   
          Satsuma             88600.000000   90966.666667   90333.333333   
          Irvington           72466.666667   74100.000000   75700.000000   
          Mount Olive        106500.000000  107333.333333  106966.666667   
          Dauphin Island     177700.000000  176466.666667  177133.333333   
          Tuscaloosa         104500.000000  106033.333333  108500.000000   
          Pleasant Grove     110000.000000  110233.333333  109000.000000   
          Edgewater                    NaN            NaN            NaN   
          Robertsdale         86833.333333   87666.666667   89000.000000   
          Elmore                       NaN            NaN            NaN   
          Crane Hill                   NaN            NaN            NaN   
          Morris             124066.666667  124500.000000  124233.333333   
          Guntersville        92933.333333   93500.000000   93600.000000   
          Hope Hull          111666.666667  111800.000000  112966.666667   
          Elberta            134700.000000  147033.333333  147166.666667   
          Alabaster          122133.333333  123066.666667  123166.666667   
          Holly Pond                   NaN            NaN            NaN   
          Mount Vernon        49533.333333   52433.333333   51200.000000   
          Union Grove         87500.000000   85566.666667   85233.333333   
          Hoover             184966.666667  186366.666667  187200.000000   
          Center Point        80966.666667   81233.333333   81500.000000   
          Seminole           113733.333333  123300.000000  128366.666667   
          Chalkville          94100.000000   94433.333333   94433.333333   
          Priceville                   NaN            NaN            NaN   
          Hazel Green                  NaN            NaN            NaN   
          Summerdale          91266.666667   91433.333333   96633.333333   
          Calera             108933.333333  110366.666667  108000.000000   
          Vance               99900.000000  103100.000000  105366.666667   
          Ralph              124166.666667  126766.666667  129233.333333   
...                                    ...            ...            ...   
Wisconsin Menomonee Falls    157733.333333  154533.333333  152366.666667   
          Brookfield         198700.000000  195333.333333  192866.666667   
          Oak Creek           96066.666667   99400.000000  104733.333333   
          Barton                       NaN            NaN            NaN   
          Roxbury            188800.000000  195866.666667  199233.333333   
          Cambria             90433.333333   91133.333333   91266.666667   
          Geneva              95466.666667   88600.000000   84433.333333   
          Sturtevant          79466.666667   77633.333333   78666.666667   
          Medina             173066.666667  176233.333333  176166.666667   
          Osborn              98333.333333   96200.000000   98666.666667   
          Sugar Creek         74166.666667   69100.000000   66066.666667   
          Utica              101600.000000  104000.000000  109700.000000   
          Rock                98933.333333  100366.666667   99900.000000   
          Eaton              110766.666667  114933.333333  121466.666667   
          Town of Eagle      169166.666667  167266.666667  165333.333333   
          Wales                        NaN            NaN            NaN   
          Lisbon             199733.333333  194700.000000  192133.333333   
          Holmen                       NaN            NaN            NaN   
          Holland            151033.333333  150500.000000  153233.333333   
          Lawrence           124066.666667  128233.333333  134466.666667   
        

In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    rstart=get_recession_start()
    rend=get_recession_end()
    rbottom=get_recession_bottom()
    
    house_range=house.loc[:,rstart:rend].reset_index()
    
    #state_names=pd.unique(utowns['State'].values)
    #region_names=pd.unique(utowns['RegionName'].values)
    #sr_names=utowns.values
    
    utowns['isTown']=True
    house_range=pd.merge(house_range,utowns,how='left',left_on=['State','RegionName'],right_on=['State','RegionName'])
    house_range['decline']=house_range[rstart]-house_range[rbottom]
    
    #house_range=pd.merge(house,utowns,how='inner',left_index=True,right_on=['State','RegionName'])
    
    house_utown=house_range[house_range['isTown'] == True]['decline']
    house_other=house_range[house_range['isTown'] != True]['decline']
    
    t,p=ttest_ind(house_utown.dropna(),house_other.dropna())    
    different=p<0.01
    
    if house_utown.mean() < house_other.mean(): # lower price decline
        better="university town"
    else:
        better="non-university town"

    return different,p,better

run_ttest()

(True, 0.0043252148535112009, 'university town')